# Tool-Calling Fine-Tuning with SFT

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ProfSynapse/Toolset-Training/blob/main/Trainers/notebooks/sft_colab_tool_calling.ipynb)

## 🎓 What You'll Learn

This notebook teaches you how to fine-tune a language model to use the **Claudesidian-MCP toolset** for Obsidian vault operations. By the end, you'll have:

- **A custom AI model** that can call tools to manage Obsidian vaults
- **Hands-on experience** with supervised fine-tuning (SFT)
- **Understanding** of hyperparameters and how they affect training

## 🔬 What is SFT?

**SFT (Supervised Fine-Tuning)** is like teaching through examples:
- You show the model examples of correct tool-calling behavior
- The model learns to replicate those patterns
- Use SFT when teaching a model a **new skill** (like using tools)

**When to use SFT:**
- ✅ Teaching tool-calling from scratch
- ✅ Learning new task formats
- ✅ Initial training with positive examples

**Not for:**
- ❌ Refining existing behavior (use KTO instead)
- ❌ Teaching preferences between good/bad outputs (use preference learning)

## 💻 Hardware Requirements

**Recommended GPU:**
- 7B models: T4 (15GB VRAM) - ✅ **Free Colab tier works!**
- 13B models: A100 (40GB VRAM) - Colab Pro
- 70B models: A100 (80GB VRAM) - Colab Pro+

**Training time:** ~45 minutes for a 7B model

## 1. Installation

Install Unsloth and dependencies. This takes ~2 minutes.

In [ ]:
# Install Unsloth for faster training
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
# Install training dependencies
%%capture
!pip install -U "transformers>=4.45.0"
!pip install "datasets==4.3.0"
!pip install -U accelerate bitsandbytes
!pip install -U trl peft xformers triton

## 2. Mount Google Drive (Optional)

Save checkpoints to Google Drive so they persist if runtime disconnects.

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

# Create output directory
DRIVE_OUTPUT_DIR = "/content/drive/MyDrive/SFT_Training"
os.makedirs(DRIVE_OUTPUT_DIR, exist_ok=True)

print(f"✓ Google Drive mounted")
print(f"✓ Checkpoints will be saved to: {DRIVE_OUTPUT_DIR}")

## 3. HuggingFace Credentials

Add your HF token to Colab secrets:
1. Click the 🔑 key icon in the left sidebar
2. Add new secret: `HF_TOKEN`
3. Get token from https://huggingface.co/settings/tokens

In [ ]:
import os
from google.colab import userdata
from huggingface_hub import HfApi

# Get token from Colab secrets
HF_TOKEN = userdata.get('HF_TOKEN')
os.environ['HF_TOKEN'] = HF_TOKEN

# Get your HuggingFace username automatically
api = HfApi()
hf_user = api.whoami(token=HF_TOKEN)["name"]

print(f"✓ HuggingFace token loaded")
print(f"✓ Username: {hf_user}")

## 4. Model Configuration

**What this does:** Choose the base model you want to fine-tune and configure basic settings.

Think of this like choosing which "brain" you want to teach tool-calling skills to.

In [ ]:
"""
═══════════════════════════════════════════════════════════════════════════
BASE MODEL SELECTION - Choose Your Foundation Model
═══════════════════════════════════════════════════════════════════════════
All models below are 4-bit quantized to work on limited VRAM.
Larger models = better quality but slower training and more memory needed.

📚 Browse all models: https://huggingface.co/unsloth
📖 Documentation: https://docs.unsloth.ai/get-started/all-our-models
"""

# 🎯 MODEL_NAME: The base model to fine-tune
#    What it is: The starting "brain" before we teach it tool-calling
#
MODEL_NAME = "unsloth/mistral-7b-v0.3-bnb-4bit"  # ← Default: Recommended for free Colab!


# ════════════════════════════════════════════════════════════════════════════
# 🔬 TINY MODELS (0.5B-1B) - Ultra-fast experimentation
# ════════════════════════════════════════════════════════════════════════════
# Perfect for: Rapid prototyping, testing pipelines, minimal hardware
# VRAM: ~2-4 GB | Training time: ~10 minutes | Quality: Basic

# Qwen 2.5 - Smallest viable models
# MODEL_NAME = "unsloth/Qwen2.5-0.5B-Instruct-bnb-4bit"        # 0.5B: Ultra-tiny
# MODEL_NAME = "unsloth/Qwen2.5-1.5B-Instruct-bnb-4bit"        # 1.5B: Better quality

# Llama 3.2 - Meta's smallest
# MODEL_NAME = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"        # 1B: Minimal Llama

# DeepSeek R1 Distilled - Reasoning capability in tiny package
# MODEL_NAME = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-bnb-4bit"  # Distilled reasoning


# ════════════════════════════════════════════════════════════════════════════
# 📦 SMALL MODELS (3B-4B) - Fast iteration, free Colab compatible
# ════════════════════════════════════════════════════════════════════════════
# Perfect for: Quick experiments, limited hardware, fast training (~20-30 min)
# VRAM: ~6-8 GB | Training time: ~25 minutes | Quality: Good for simple tasks

# Llama 3.2 (Meta) - Latest small Llama, good reasoning
# MODEL_NAME = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"        # 3B: Excellent instruction following
# MODEL_NAME = "unsloth/Llama-3.2-3B-bnb-4bit"                 # 3B base (not instruct-tuned)

# Qwen 2.5 (Alibaba) - Strong coding & reasoning
# MODEL_NAME = "unsloth/Qwen2.5-3B-Instruct-bnb-4bit"          # 3B: Best-in-class small model
# MODEL_NAME = "unsloth/Qwen2.5-3B-bnb-4bit"                   # 3B base
# MODEL_NAME = "unsloth/Qwen2.5-Coder-3B-Instruct-bnb-4bit"    # 3B: Optimized for code

# Phi-3 (Microsoft) - Efficient small models
# MODEL_NAME = "unsloth/Phi-3.5-mini-instruct-bnb-4bit"        # 3.8B: Microsoft's efficient model
# MODEL_NAME = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit"       # 3.8B: Original Phi-3

# Gemma 2 (Google) - Memory efficient architecture
# MODEL_NAME = "unsloth/gemma-2-2b-it-bnb-4bit"                # 2B: Very fast, decent quality
# MODEL_NAME = "unsloth/gemma-2-2b-bnb-4bit"                   # 2B base

# DeepSeek R1 Distilled - Reasoning in small package
# MODEL_NAME = "unsloth/DeepSeek-R1-Distill-Qwen-3B-bnb-4bit"  # 3B: Distilled reasoning


# ════════════════════════════════════════════════════════════════════════════
# 🚀 MEDIUM MODELS (7B-9B) - Best balance, production ready
# ════════════════════════════════════════════════════════════════════════════
# Perfect for: Production use, good quality, free Colab compatible (~45 min)
# VRAM: ~9-11 GB | Training time: ~45 minutes | Quality: Excellent ⭐ RECOMMENDED

# Mistral 7B v0.3 (Mistral AI) - Industry standard, reliable
# MODEL_NAME = "unsloth/mistral-7b-v0.3-bnb-4bit"              # ⭐ RECOMMENDED: Best all-around 7B
# MODEL_NAME = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"     # Pre-instruct tuned version
# MODEL_NAME = "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit"   # 12B: Nemo variant (larger context)
# MODEL_NAME = "unsloth/Mistral-Nemo-Base-2407-bnb-4bit"       # 12B base

# Llama 3.1 (Meta) - Latest from Meta, strong performance
# MODEL_NAME = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"   # 8B: Meta's flagship
# MODEL_NAME = "unsloth/Meta-Llama-3.1-8B-bnb-4bit"            # 8B base (not instruct-tuned)

# Llama 3.3 (Meta) - Newest Llama family
# MODEL_NAME = "unsloth/Llama-3.3-70B-Instruct-bnb-4bit"       # 70B: Requires A100 80GB

# Llama 3 (Meta) - Proven stable release
# MODEL_NAME = "unsloth/llama-3-8b-Instruct-bnb-4bit"          # 8B: Proven and stable

# Qwen 2.5 (Alibaba) - Top-tier reasoning and coding
# MODEL_NAME = "unsloth/Qwen2.5-7B-Instruct-bnb-4bit"          # 7B: Excellent all-around
# MODEL_NAME = "unsloth/Qwen2.5-7B-bnb-4bit"                   # 7B base
# MODEL_NAME = "unsloth/Qwen2.5-Coder-7B-Instruct-bnb-4bit"    # 7B: Specialized for coding
# MODEL_NAME = "unsloth/Qwen2.5-Coder-7B-bnb-4bit"             # 7B code base

# Gemma 2 (Google) - Memory efficient
# MODEL_NAME = "unsloth/gemma-2-9b-it-bnb-4bit"                # 9B: Memory-efficient architecture
# MODEL_NAME = "unsloth/gemma-2-9b-bnb-4bit"                   # 9B base

# DeepSeek R1 Distilled - Reasoning capability
# MODEL_NAME = "unsloth/DeepSeek-R1-Distill-Qwen-7B-bnb-4bit"  # 7B: Distilled reasoning
# MODEL_NAME = "unsloth/DeepSeek-R1-Distill-Llama-8B-bnb-4bit" # 8B: Llama-based reasoning


# ════════════════════════════════════════════════════════════════════════════
# 💎 LARGE MODELS (13B-20B) - High quality, needs Colab Pro (A100 40GB)
# ════════════════════════════════════════════════════════════════════════════
# Perfect for: Maximum quality, complex tasks, production deployment
# VRAM: ~14-20 GB | Training time: ~90-120 minutes | Quality: Excellent

# Llama 2 (Meta) - Proven, stable
# MODEL_NAME = "unsloth/llama-2-13b-bnb-4bit"                  # 13B: Reliable base
# MODEL_NAME = "unsloth/llama-2-13b-chat-bnb-4bit"             # 13B: Chat-optimized

# Qwen 2.5 (Alibaba) - Best 14B model available
# MODEL_NAME = "unsloth/Qwen2.5-14B-Instruct-bnb-4bit"         # 14B: Top performance
# MODEL_NAME = "unsloth/Qwen2.5-14B-bnb-4bit"                  # 14B base
# MODEL_NAME = "unsloth/Qwen2.5-Coder-14B-Instruct-bnb-4bit"   # 14B: Code specialist
# MODEL_NAME = "unsloth/Qwen2.5-Coder-14B-bnb-4bit"            # 14B code base

# CodeLlama (Meta) - Specialized for code generation
# MODEL_NAME = "unsloth/codellama-13b-bnb-4bit"                # 13B: Code-focused

# GPT-OSS (OpenAI open weights) - OpenAI architecture
# MODEL_NAME = "unsloth/gpt-oss-20b-unsloth-bnb-4bit"          # 20B: OpenAI's open model

# DeepSeek R1 Distilled - Reasoning models
# MODEL_NAME = "unsloth/DeepSeek-R1-Distill-Qwen-14B-bnb-4bit" # 14B: Strong reasoning


# ════════════════════════════════════════════════════════════════════════════
# 🏆 EXTRA-LARGE MODELS (24B-32B) - Best quality, needs A100 80GB
# ════════════════════════════════════════════════════════════════════════════
# Perfect for: Maximum capability, specialized tasks, research
# VRAM: ~20-30 GB | Training time: ~2-3 hours | Quality: State-of-the-art

# Mistral Small (Mistral AI) - Latest from Mistral
# MODEL_NAME = "unsloth/Mistral-Small-24B-Instruct-2501-bnb-4bit"  # 24B: Newest Mistral

# Qwen 2.5 (Alibaba) - Massive models
# MODEL_NAME = "unsloth/Qwen2.5-32B-Instruct-bnb-4bit"         # 32B: Powerhouse
# MODEL_NAME = "unsloth/Qwen2.5-32B-bnb-4bit"                  # 32B base
# MODEL_NAME = "unsloth/Qwen2.5-Coder-32B-Instruct-bnb-4bit"   # 32B: Code specialist
# MODEL_NAME = "unsloth/Qwen2.5-Coder-32B-bnb-4bit"            # 32B code base

# Gemma 2 (Google) - Large variant
# MODEL_NAME = "unsloth/gemma-2-27b-it-bnb-4bit"               # 27B: Largest Gemma
# MODEL_NAME = "unsloth/gemma-2-27b-bnb-4bit"                  # 27B base

# DeepSeek R1 Distilled - Large reasoning models
# MODEL_NAME = "unsloth/DeepSeek-R1-Distill-Qwen-32B-bnb-4bit" # 32B: Maximum reasoning


# ════════════════════════════════════════════════════════════════════════════
# 🌟 MASSIVE MODELS (70B-405B) - Multi-GPU or highest-end hardware
# ════════════════════════════════════════════════════════════════════════════
# Perfect for: Research, benchmarking, specialized high-stakes applications
# VRAM: ~40-80+ GB | Training time: ~4-8+ hours | Quality: Frontier

# Llama 3.3 (Meta) - Latest flagship
# MODEL_NAME = "unsloth/Llama-3.3-70B-Instruct-bnb-4bit"       # 70B: Latest Llama

# Llama 3.1 (Meta) - Proven large models
# MODEL_NAME = "unsloth/Meta-Llama-3.1-70B-Instruct-bnb-4bit"  # 70B: Proven quality
# MODEL_NAME = "unsloth/Meta-Llama-3.1-70B-bnb-4bit"           # 70B base
# MODEL_NAME = "unsloth/Meta-Llama-3.1-405B-Instruct-bnb-4bit" # 405B: MASSIVE (multi-GPU required)
# MODEL_NAME = "unsloth/Meta-Llama-3.1-405B-bnb-4bit"          # 405B base

# Llama 3 (Meta) - Previous generation large
# MODEL_NAME = "unsloth/llama-3-70b-Instruct-bnb-4bit"         # 70B: Proven stable

# Llama 2 (Meta) - Earlier generation
# MODEL_NAME = "unsloth/llama-2-70b-bnb-4bit"                  # 70B: Older but reliable
# MODEL_NAME = "unsloth/llama-2-70b-chat-bnb-4bit"             # 70B chat

# Qwen 2.5 (Alibaba) - Massive reasoning/coding
# MODEL_NAME = "unsloth/Qwen2.5-72B-Instruct-bnb-4bit"         # 72B: Excellent quality
# MODEL_NAME = "unsloth/Qwen2.5-72B-bnb-4bit"                  # 72B base

# CodeLlama (Meta) - Large code models
# MODEL_NAME = "unsloth/codellama-34b-bnb-4bit"                # 34B: Strong coding
# MODEL_NAME = "unsloth/codellama-70b-bnb-4bit"                # 70B: Maximum code capability

# DeepSeek V3 & R1 - Frontier reasoning models
# MODEL_NAME = "unsloth/DeepSeek-V3-bnb-4bit"                  # 671B: Frontier model (multi-GPU)
# MODEL_NAME = "unsloth/DeepSeek-R1-bnb-4bit"                  # 671B: Reasoning specialist
# MODEL_NAME = "unsloth/DeepSeek-R1-Distill-Llama-70B-bnb-4bit" # 70B: Distilled reasoning

# Qwen QwQ (Reasoning specialist)
# MODEL_NAME = "unsloth/QwQ-32B-Preview-bnb-4bit"              # 32B: Reasoning-focused


# ════════════════════════════════════════════════════════════════════════════
# 🎨 SPECIALIZED MODELS - Vision, Medical, Reasoning
# ════════════════════════════════════════════════════════════════════════════

# Vision Models (Multimodal)
# MODEL_NAME = "unsloth/Qwen2-VL-2B-Instruct-bnb-4bit"         # 2B: Vision + language
# MODEL_NAME = "unsloth/Qwen2-VL-7B-Instruct-bnb-4bit"         # 7B: Vision + language
# MODEL_NAME = "unsloth/Pixtral-12B-2409-bnb-4bit"             # 12B: Mistral's vision model

# Medical Models (Healthcare specialist)
# MODEL_NAME = "unsloth/gemma-2-9b-medgemma-bnb-4bit"          # 9B: Medical Gemma

# Reasoning Models (Advanced problem-solving)
# MODEL_NAME = "unsloth/Phi-4-reasoning-bnb-4bit"              # 14B: Microsoft's reasoning
# MODEL_NAME = "unsloth/Phi-4-reasoning-plus-bnb-4bit"         # 14B+: Enhanced reasoning


# ════════════════════════════════════════════════════════════════════════════
# 🔧 OTHER MODEL FAMILIES
# ════════════════════════════════════════════════════════════════════════════

# Phi-4 (Microsoft) - Latest efficient models
# MODEL_NAME = "unsloth/Phi-4-bnb-4bit"                        # 14B: Microsoft's latest

# Phi-3 (Microsoft) - Efficient architecture
# MODEL_NAME = "unsloth/Phi-3-medium-4k-instruct-bnb-4bit"     # 14B: Medium variant

# Mixtral (Mistral AI) - Mixture of Experts
# MODEL_NAME = "unsloth/Mixtral-8x7B-Instruct-v0.1-bnb-4bit"   # 47B params, 13B active

# Gemma 1 (Google) - Earlier generation
# MODEL_NAME = "unsloth/gemma-7b-it-bnb-4bit"                  # 7B: Gemma 1
# MODEL_NAME = "unsloth/gemma-7b-bnb-4bit"                     # 7B base

# GLM-4 (Zhipu AI)
# MODEL_NAME = "unsloth/glm-4-9b-chat-bnb-4bit"                # 9B: Chinese-English bilingual


# ════════════════════════════════════════════════════════════════════════════
# 📏 MAX_SEQ_LENGTH: How much context the model can "see"
# ════════════════════════════════════════════════════════════════════════════

# 🎯 MAX_SEQ_LENGTH: Maximum conversation length in tokens
#    What it is: How much text the model can process at once
#    Trade-offs:
#      - 1024: Minimal memory, fastest training, good for short interactions
#      - 2048: Balanced, handles most conversations, recommended ⭐
#      - 4096: Long context, uses more VRAM, slower training
#      - 8192+: Very long context, needs significant VRAM
#
MAX_SEQ_LENGTH = 2048  # ← Recommended: Works for 95% of tool-calling examples


# ════════════════════════════════════════════════════════════════════════════
# 📊 DATASET CONFIGURATION
# ════════════════════════════════════════════════════════════════════════════

# 🎯 DATASET: Pre-built training data for tool-calling
#    What it is: ~5,500 high-quality examples of correct tool usage
#    Format: User request → Tool call → Result → Assistant response
#
DATASET_NAME = "professorsynapse/claudesidian-synthetic-dataset"
DATASET_FILE = "syngen_tools_sft_11.22.25.jsonl"  # Latest version (11/22/25)

# Note: You can upload your own dataset to HuggingFace and use it here!
# Just change DATASET_NAME to "your-username/your-dataset-name"


# ════════════════════════════════════════════════════════════════════════════
# 🏷️ OUTPUT MODEL NAME
# ════════════════════════════════════════════════════════════════════════════

# 🎯 OUTPUT_MODEL_NAME: What to call your trained model on HuggingFace
#    Pick something descriptive that indicates:
#    - What it does (e.g., "tool-calling", "obsidian-assistant")
#    - Model size (e.g., "7b", "mistral-7b")
#    - Version (e.g., "v1", "2024-11")
#
OUTPUT_MODEL_NAME = "nexus-tools-sft-7b"  # ← Customize this!

# Good naming examples:
# - "obsidian-tools-mistral-7b"
# - "claudesidian-assistant-v2"
# - "tool-calling-llama-8b"


# ════════════════════════════════════════════════════════════════════════════
# ✅ Print Configuration Summary
# ════════════════════════════════════════════════════════════════════════════
print("╔" + "═" * 78 + "╗")
print("║" + " " * 25 + "TRAINING CONFIGURATION" + " " * 31 + "║")
print("╠" + "═" * 78 + "╣")
print(f"║  Model:        {MODEL_NAME:<60} ║")
print(f"║  Max Length:   {MAX_SEQ_LENGTH:<60} tokens ║")
print(f"║  Dataset:      {DATASET_NAME}/{DATASET_FILE:<33} ║")
print(f"║  Output:       {hf_user}/{OUTPUT_MODEL_NAME:<49} ║")
print("╠" + "═" * 78 + "╣")
print("║  Outputs on HuggingFace:                                                 ║")
print(f"║    • LoRA adapters:    {hf_user}/{OUTPUT_MODEL_NAME:<40} ║")
print(f"║    • Merged 16-bit:    {hf_user}/{OUTPUT_MODEL_NAME}-merged{' ' * (33 - len(OUTPUT_MODEL_NAME))} ║")
print("╚" + "═" * 78 + "╝")

## 5. Load Model and Tokenizer

**What this does:** Downloads the base model and prepares it for training.

The model is the "brain" that will learn tool-calling. The tokenizer converts text into numbers the model can process. We use 4-bit quantization to fit large models into limited GPU memory.

In [ ]:
from unsloth import FastLanguageModel
import torch

# Check GPU
print(f"Using GPU: {torch.cuda.get_device_name(0)}")
print(f"CUDA version: {torch.version.cuda}")
print(f"Available VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
print()

In [ ]:
# Load the base model and tokenizer from HuggingFace
# This downloads the model weights (~7GB for 7B models)
# 
# Parameters explained:
#   model_name: Which model to download
#   max_seq_length: Max tokens model can process at once
#   dtype=None: Auto-detect best precision for your GPU
#   load_in_4bit=True: Use 4-bit quantization to save memory
#   token: Your HF token for accessing the model

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,  # Auto-detect (usually bfloat16 or float16)
    load_in_4bit=True,  # Reduces memory usage by ~75%
    token=HF_TOKEN,
)

print("✓ Model loaded successfully")
print(f"  Model has {sum(p.numel() for p in model.parameters()):,} parameters")

## 6. Apply LoRA Adapters

**What this does:** Add trainable "adapter" layers to the model instead of training the entire thing.

Think of LoRA like teaching a new skill through muscle memory - we add small specialized layers that learn the new behavior, while keeping the main "brain" frozen. This is way faster and uses less memory than retraining everything.

In [ ]:
"""
═══════════════════════════════════════════════════════════════════════════
LORA CONFIGURATION - How the model learns
═══════════════════════════════════════════════════════════════════════════
These parameters control how we add trainable "adapter" layers to the model.
"""

model = FastLanguageModel.get_peft_model(
    model,
    
    # 🎯 r (rank): Size of the adapter matrices
    #    What it controls: Learning capacity vs memory usage
    #    Turn UP (64, 128): More capacity to learn complex patterns, uses more VRAM
    #    Turn DOWN (8, 16): Faster, less memory, but might not capture all nuances
    #    Sweet spot (32): Good balance for most tasks
    r=32,
    
    # 🎯 lora_alpha: Scaling factor for LoRA updates
    #    What it controls: How much the adapters influence the model
    #    Turn UP (128): Stronger influence from fine-tuning (be careful of overfitting!)
    #    Turn DOWN (16): More conservative, preserves base model behavior
    #    Rule of thumb: Usually set to 2x the rank (if r=32, use alpha=64)
    lora_alpha=64,
    
    # 🎯 lora_dropout: Regularization to prevent overfitting
    #    What it controls: Randomly drops connections during training
    #    Turn UP (0.1): More regularization, helps if model memorizes too much
    #    Turn DOWN (0.0): No regularization, use if you have lots of diverse data
    #    Sweet spot (0.05): Light regularization for most cases
    lora_dropout=0.05,
    
    # 🎯 bias: Whether to train bias parameters
    #    Options: "none" (don't train), "all" (train all), "lora_only" (train LoRA biases)
    #    "none" is standard and works well for most cases
    bias="none",
    
    # 🎯 target_modules: Which parts of the model to add adapters to
    #    What it controls: Where learning happens in the neural network
    #    These are the "attention" and "feed-forward" layers
    #    More modules = more capacity but slower training
    #    This list covers all the key layers for language understanding
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    
    # 🎯 use_gradient_checkpointing: Memory optimization technique
    #    What it controls: Trade computation for memory
    #    "unsloth": Fast checkpointing (recommended)
    #    False: Faster but uses more VRAM
    use_gradient_checkpointing="unsloth",
    
    # 🎯 random_state: Random seed for reproducibility
    #    What it controls: Ensures same initialization each run
    #    Change this number to get different random starting points
    random_state=3407,
)

print("✓ LoRA adapters applied")
print(f"  Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
print(f"  Total parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"  Percentage trainable: {100 * sum(p.numel() for p in model.parameters() if p.requires_grad) / sum(p.numel() for p in model.parameters()):.2f}%")

## 7. Load and Prepare Dataset

**What this does:** Downloads training examples and formats them for the model.

The dataset contains ~5,500 examples of correct tool-calling behavior. Think of it like a textbook full of solved problems that the model will learn from.

In [ ]:
from datasets import load_dataset

"""
═══════════════════════════════════════════════════════════════════════════
LOAD DATASET FROM HUGGINGFACE
═══════════════════════════════════════════════════════════════════════════
This downloads pre-made training examples of correct tool-calling behavior.
Each example shows: user request → tool call → result → assistant response
"""

print(f"Loading dataset: {DATASET_NAME}")
dataset = load_dataset(
    DATASET_NAME,  # HuggingFace repository containing the dataset
    data_files=DATASET_FILE,  # Specific JSONL file to use
    split="train"  # Use the training split
)

print(f"✓ Loaded {len(dataset)} examples")
print(f"\nSample example:")
print(dataset[0])

"""
═══════════════════════════════════════════════════════════════════════════
SET CHAT TEMPLATE (Model-Specific)
═══════════════════════════════════════════════════════════════════════════
The chat template defines how conversations are formatted for the model.
Different models use different formats - we auto-detect and use the right one!
"""

# Mistral-specific template (uses [INST] format)
MISTRAL_CHAT_TEMPLATE = """{{ bos_token }}{% for message in messages %}{% if message['role'] == 'system' %}{% if loop.index == 1 %}{{ message['content'] + ' ' }}{% endif %}{% elif message['role'] == 'user' %}{{ '[INST] ' + message['content'] + ' [/INST]' }}{% elif message['role'] == 'assistant' %}{{ ' ' + message['content'] + eos_token }}{% endif %}{% endfor %}"""

# Generic fallback template for other models
DEFAULT_CHAT_TEMPLATE = """{% for message in messages %}{% if message['role'] == 'user' %}{{ '<|user|>\\n' + message['content'] + eos_token }}{% elif message['role'] == 'system' %}{{ '<|system|>\\n' + message['content'] + eos_token }}{% elif message['role'] == 'assistant' %}{{ '<|assistant|>\\n'  + message['content'] + eos_token }}{% endif %}{% if loop.last and add_generation_prompt %}{{ '<|assistant|>' }}{% endif %}{% endfor %}"""

if tokenizer.chat_template is None:
    # Detect model type and use appropriate template
    is_mistral = 'mistral' in MODEL_NAME.lower()
    
    if is_mistral:
        print("\n✓ Detected Mistral model - using [INST] format")
        tokenizer.chat_template = MISTRAL_CHAT_TEMPLATE
        print("   Format: <s>[INST] user [/INST] assistant</s>")
        print("   This is the official Mistral chat format!")
    else:
        print("\n⚠️  No chat template found, using default format")
        tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE
        print("   Format: <|user|>\\ncontent</s>")
else:
    print("\n✓ Tokenizer already has chat template")
    if 'mistral' in MODEL_NAME.lower():
        print("   Using Mistral model - template should use [INST] format")

In [ ]:
"""
═══════════════════════════════════════════════════════════════════════════
FORMAT DATASET FOR TRAINING
═══════════════════════════════════════════════════════════════════════════
Convert the conversation format into the exact text format the model expects.
This applies the chat template to each example.
"""

def format_chat_template(example):
    """
    Convert conversations to tokenizer's chat template.
    
    Input: {"conversations": [{"role": "user", "content": "..."}, ...]}
    Output: {"text": "<|im_start|>user\n...<|im_end|>\n..."}
    """
    text = tokenizer.apply_chat_template(
        example["conversations"],  # List of message dicts
        tokenize=False,  # Return string, not token IDs
        add_generation_prompt=False  # Don't add prompt for model to continue
    )
    return {"text": text}

# Apply formatting to entire dataset
# This creates a new "text" field with formatted conversations
dataset = dataset.map(
    format_chat_template,  # Function to apply
    remove_columns=dataset.column_names,  # Remove original columns
    desc="Formatting dataset"  # Progress bar description
)

print("✓ Dataset formatted for training")
print(f"\nFormatted example (first 500 characters):")
print(dataset[0]["text"][:500])
print("\n💡 The full formatted text will be used during training")

## 8. Training Configuration

**What this does:** Set the hyperparameters that control how the model learns.

This is the **most important section** - these settings determine how fast the model learns, how much memory it uses, and how good the final result will be. Think of it like configuring a study plan: how many hours per day, how many review sessions, how to handle difficult material, etc.

In [ ]:
from trl import SFTTrainer, SFTConfig
from unsloth import is_bfloat16_supported
from datetime import datetime

# Create timestamped output directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"{DRIVE_OUTPUT_DIR}/{timestamp}"

"""
═══════════════════════════════════════════════════════════════════════════
TRAINING HYPERPARAMETERS - How the model learns
═══════════════════════════════════════════════════════════════════════════
These control the learning process. Getting these right is crucial!
"""

training_args = SFTConfig(
    output_dir=output_dir,
    
    # ──────────────────────────────────────────────────────────────────────
    # BATCH SIZE & MEMORY SETTINGS
    # ──────────────────────────────────────────────────────────────────────
    
    # 🎯 per_device_train_batch_size: Examples processed at once
    #    What it controls: How many training examples fit in GPU memory at once
    #    Turn UP (4, 8): Faster training, but uses more VRAM (might run out of memory!)
    #    Turn DOWN (1): Slower but uses less memory (use if you get OOM errors)
    #    Sweet spot (2): Works on most free Colab GPUs
    per_device_train_batch_size=2,
    
    # 🎯 gradient_accumulation_steps: Simulate larger batches
    #    What it controls: How many mini-batches before updating weights
    #    Turn UP (8, 16): Simulates larger batch, more stable training
    #    Turn DOWN (1, 2): Faster updates, but noisier learning
    #    Effective batch = per_device_batch_size × gradient_accumulation_steps
    #    Example: 2 × 4 = 8 effective batch size
    gradient_accumulation_steps=4,
    
    # ──────────────────────────────────────────────────────────────────────
    # LEARNING RATE & OPTIMIZATION
    # ──────────────────────────────────────────────────────────────────────
    
    # 🎯 learning_rate: How fast the model learns
    #    What it controls: Size of weight updates during training
    #    Turn UP (5e-4): Faster learning, but might overshoot optimal values
    #    Turn DOWN (1e-5): More careful learning, but takes longer
    #    Sweet spot (2e-4): Standard for SFT (100x higher than preference learning!)
    #    NOTE: This is MUCH higher than KTO learning rates (2e-7)
    learning_rate=2e-4,
    
    # 🎯 max_grad_norm: Gradient clipping to prevent exploding gradients
    #    What it controls: Maximum size of weight updates
    #    Turn UP (2.0): Allows larger updates (might cause instability)
    #    Turn DOWN (0.5): More conservative (slower learning)
    #    Sweet spot (1.0): Prevents extreme updates while allowing learning
    max_grad_norm=1.0,
    
    # 🎯 lr_scheduler_type: How learning rate changes over time
    #    What it controls: Learning rate adjustment strategy
    #    "cosine": Starts high, smoothly decreases (recommended)
    #    "linear": Decreases linearly
    #    "constant": Never changes (simpler but less effective)
    lr_scheduler_type="cosine",
    
    # 🎯 warmup_ratio: Fraction of training with increasing learning rate
    #    What it controls: How long to "warm up" before full learning rate
    #    Turn UP (0.2): Longer warmup, more stable early training
    #    Turn DOWN (0.0): No warmup, jumps straight to full LR
    #    Sweet spot (0.1): 10% of training spent warming up
    warmup_ratio=0.1,
    
    # ──────────────────────────────────────────────────────────────────────
    # TRAINING DURATION
    # ──────────────────────────────────────────────────────────────────────
    
    # 🎯 num_train_epochs: How many times to go through the entire dataset
    #    What it controls: Total training time and learning opportunity
    #    Turn UP (5, 10): More learning, but risk of overfitting (memorization)
    #    Turn DOWN (1): Faster but might not learn everything
    #    Sweet spot (3): Standard for SFT - enough to learn without memorizing
    #    NOTE: KTO only uses 1 epoch because it's for refinement, not initial learning
    num_train_epochs=3,
    
    # ──────────────────────────────────────────────────────────────────────
    # DATA HANDLING
    # ──────────────────────────────────────────────────────────────────────
    
    # 🎯 max_seq_length: Maximum tokens per example (matches earlier config)
    max_seq_length=MAX_SEQ_LENGTH,
    
    # 🎯 packing: Whether to combine short examples into one sequence
    #    What it controls: Efficiency of sequence processing
    #    True: Packs multiple short examples together (more efficient)
    #    False: Each example is separate (simpler, safer)
    #    Set to False for tool-calling to keep examples independent
    packing=False,
    
    # 🎯 dataset_text_field: Which field in dataset contains the text
    #    This tells the trainer where to find the formatted conversations
    dataset_text_field="text",
    
    # ──────────────────────────────────────────────────────────────────────
    # PRECISION & OPTIMIZATION
    # ──────────────────────────────────────────────────────────────────────
    
    # 🎯 fp16 / bf16: Mixed precision training (speed + memory optimization)
    #    What it controls: Number precision during training
    #    bf16 (bfloat16): Better for newer GPUs (A100, T4), more stable
    #    fp16 (float16): For older GPUs, slightly less stable
    #    Auto-detect: Uses bf16 if supported, fp16 otherwise
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    
    # 🎯 optim: Optimizer algorithm
    #    What it controls: How weight updates are calculated
    #    "adamw_8bit": Memory-efficient version of AdamW (recommended)
    #    "adamw_torch": Standard AdamW (uses more memory)
    #    "sgd": Simpler but less effective for LLMs
    optim="adamw_8bit",
    
    # 🎯 gradient_checkpointing: Save memory by recomputing during backward pass
    #    What it controls: Memory vs speed tradeoff
    #    True: Uses less memory (~30% reduction), slightly slower
    #    False: Faster but uses more memory
    gradient_checkpointing=True,
    
    # ──────────────────────────────────────────────────────────────────────
    # LOGGING & CHECKPOINTING
    # ──────────────────────────────────────────────────────────────────────
    
    # 🎯 logging_steps: How often to print training progress
    #    What it controls: Frequency of console updates
    #    Turn UP (10, 20): Less frequent updates, cleaner logs
    #    Turn DOWN (1): See every step (very verbose!)
    #    Sweet spot (5): Regular updates without spam
    logging_steps=5,
    
    # 🎯 save_steps: How often to save checkpoint
    #    What it controls: Backup frequency during training
    #    Turn UP (200, 500): Less frequent saves, saves disk space
    #    Turn DOWN (50): More backups, can resume from more points
    #    Sweet spot (100): Good balance for ~45min training
    save_steps=100,
    
    # 🎯 save_total_limit: Maximum number of checkpoints to keep
    #    What it controls: Disk space used for checkpoints
    #    Turn UP (5, 10): Keep more history, uses more space
    #    Turn DOWN (1, 2): Only keep latest, saves space
    #    Sweet spot (3): Last 3 checkpoints (in case latest has issues)
    save_total_limit=3,
    
    # ──────────────────────────────────────────────────────────────────────
    # REPRODUCIBILITY
    # ──────────────────────────────────────────────────────────────────────
    
    # 🎯 seed: Random seed for reproducible results
    #    What it controls: Ensures same results across runs
    #    Change this to get different random variations
    seed=42,
    
    # 🎯 report_to: Where to log metrics (wandb, tensorboard, etc.)
    #    "none": Don't upload metrics anywhere
    #    "wandb": Log to Weights & Biases (requires setup)
    #    "tensorboard": Log locally
    report_to="none",
)

# ============================================================================
# Print configuration summary
# ============================================================================
print("✓ Training configuration ready")
print(f"  Output directory: {output_dir}")
print(f"  Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Precision: {'bf16' if is_bfloat16_supported() else 'fp16'}")
print(f"\n💡 Training will take approximately 45 minutes for a 7B model")

## 9. Initialize Trainer

**What this does:** Creates the training engine that coordinates everything.

The SFTTrainer is the orchestrator - it takes the model, dataset, and configuration, then handles all the training mechanics (gradient updates, checkpointing, logging, etc.).

In [ ]:
# Create the SFTTrainer
# This combines the model, dataset, and configuration into one training pipeline

trainer = SFTTrainer(
    model=model,  # The model with LoRA adapters
    tokenizer=tokenizer,  # For converting text to tokens
    train_dataset=dataset,  # Our formatted training examples
    args=training_args,  # All the hyperparameters we configured
)

print("✓ Trainer initialized")
print("  Ready to start training!")

## 10. Train!

**What this does:** The actual learning happens here!

The model will:
1. **Read examples** from the dataset
2. **Predict** what the response should be
3. **Compare** its prediction to the correct answer
4. **Update weights** to get closer to the correct answer
5. **Repeat** this process for 3 epochs (3 full passes through the data)

**What to expect:**
- Training takes ~45 minutes for 7B models on T4 GPU
- You'll see progress updates every 5 steps
- Loss should generally decrease over time (learning is working!)
- Checkpoints are saved every 100 steps to Google Drive

**What the metrics mean:**
- **Loss:** How "wrong" the model is (lower = better, aim for <1.0)
- **Learning Rate:** Gradually decreases as training progresses
- **Samples/sec:** Training speed (depends on GPU)

In [ ]:
# Check GPU memory
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")
print()

# Start training
print("=" * 60)
print("STARTING TRAINING")
print("=" * 60)
print()

trainer_stats = trainer.train()

print()
print("=" * 60)
print("TRAINING COMPLETED")
print("=" * 60)

## 11. Upload to HuggingFace

**What this does:** Share your trained model with the world!

We'll create **three versions** of your model:

1. **LoRA adapters** (~320MB) - Small files that contain just the changes
   - Fast to download
   - Need to be combined with base model to use
   
2. **Merged 16-bit model** (~14GB) - Full model with adapters merged in
   - High quality, no precision loss
   - Large file size
   - Best for local deployment (Ollama, LM Studio)
   
3. **GGUF quantizations** - Optimized versions for CPU/GPU inference
   - Q4_K_M (~3.5GB) - Recommended for most users
   - Q5_K_M (~4.5GB) - Better quality, larger size
   - Q8_0 (~7GB) - Nearly full quality
   - These work directly with Ollama and LM Studio!

In [ ]:
# Upload LoRA adapters
model.push_to_hub(
    f"{hf_user}/{OUTPUT_MODEL_NAME}",
    token=HF_TOKEN,
    private=False
)
tokenizer.push_to_hub(
    f"{hf_user}/{OUTPUT_MODEL_NAME}",
    token=HF_TOKEN,
    private=False
)

print(f"✓ LoRA adapters uploaded to HuggingFace")
print(f"  View at: https://huggingface.co/{hf_user}/{OUTPUT_MODEL_NAME}")

In [ ]:
# Upload merged 16-bit model
print("Merging LoRA weights into base model (16-bit)...")
print("This will take ~5 minutes...")

model.push_to_hub_merged(
    f"{hf_user}/{OUTPUT_MODEL_NAME}-merged",
    tokenizer,
    save_method="merged_16bit",
    token=HF_TOKEN,
    private=False
)

print(f"✓ Merged model uploaded to HuggingFace")
print(f"  View at: https://huggingface.co/{hf_user}/{OUTPUT_MODEL_NAME}-merged")

In [ ]:
# Create GGUF quantizations
quantization_methods = ["q4_k_m", "q5_k_m", "q8_0"]

print("Creating GGUF quantizations...")
print(f"This will create {len(quantization_methods)} versions")
print()

model.push_to_hub_gguf(
    f"{hf_user}/{OUTPUT_MODEL_NAME}",
    tokenizer,
    quantization_method=quantization_methods,
    token=HF_TOKEN,
)

print()
print("✓ GGUF quantizations created and uploaded!")
print(f"  View at: https://huggingface.co/{hf_user}/{OUTPUT_MODEL_NAME}")

## 🎉 Done!

Your model has been trained and uploaded to HuggingFace!

## 📊 What You Accomplished

✅ **Fine-tuned a 7B parameter language model** to use the Claudesidian toolset  
✅ **Learned about SFT hyperparameters** and how they affect training  
✅ **Created multiple model formats** (LoRA, merged, GGUF) for different use cases  
✅ **Published your model** to HuggingFace for others to use  

## 🚀 Next Steps

### 1. Test Your Model

**Using LM Studio:**
1. Open LM Studio → "Discover" tab
2. Search for your username: `{hf_user}`
3. Download the Q4_K_M or Q5_K_M GGUF version
4. Load and test with tool-calling prompts!

**Using Ollama:**
```bash
# Download your model
ollama pull {hf_user}/{OUTPUT_MODEL_NAME}

# Test it
ollama run {hf_user}/{OUTPUT_MODEL_NAME}
```

### 2. Evaluate Quality

Run the Evaluator to test tool-calling accuracy:
```bash
python -m Evaluator.cli \
  --model {OUTPUT_MODEL_NAME} \
  --prompt-set Evaluator/prompts/full_coverage.json \
  --output results.json
```

### 3. Refine Further (Optional)

If you want to improve your model:
- **Collect more data**: Add examples where your model struggles
- **Try KTO training**: Refine behavior by showing good vs bad examples
- **Adjust hyperparameters**: Experiment with learning rate, batch size, etc.

## 🎓 Learn More

- **KTO Training**: For preference learning (good vs bad outputs)
- **Dataset Creation**: Build custom training data for your use case
- **Model Evaluation**: Systematic testing of tool-calling accuracy

## 💡 Tips for Better Models

1. **More data is better**: 5,000+ examples produce robust models
2. **Quality over quantity**: Clean, accurate examples matter more than volume
3. **Test thoroughly**: Use the Evaluator to catch issues early
4. **Iterate**: Fine-tuning is iterative - train, test, improve, repeat

---

**Questions or issues?** Check the documentation or open an issue on GitHub!